# <font color = red> Simulation Analyze
</font>This notebook records the parameters for Wright-Fisher simulations used to generate our test data sets, as well as commands for running infernce algorithms on the test data and compiling the results. 

## Contents
- ### [Libraries and variables](#lib)
- ### Data analyze
    - #### [Generation of test data through Wright-Fisher simulations](#sim)
    - #### [Repeating simulation](#rep)
    - #### [Running the inference algorithms and compiling output](#nsdt)
    - #### [Dealing with the output](#deal)
    - #### [Supplement: multiple case and no recombination term case¶](#supp)

## <a id='lib'></a> Libraries and variables

In [1]:
# Full library list and version numbers

print('This notebook was prepared using:')

import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import random

import pandas as pd

import sklearn as sk
from sklearn.metrics import roc_auc_score

import importlib
import simulation as sim

# GitHub directories
HIV_DIR = 'data/HIV'
MPL_DIR = 'src/MPL'
SIM_DIR = 'data/simulation'
FIG_DIR = 'figures'

This notebook was prepared using:
python version 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:26:08) [Clang 14.0.6 ]
numpy version 1.24.2


## Data analyze

<a id='sim'></a>
### Single Wright-Fisher simulation example
The fitness model work like this:
$f_a = 1 + \sum_i^L s_i g_i^a + \sum_n^{N_t} s_n g_n^a$

This simulation begins with all wild type, which only has 2 alleles (wild type and mutant type).

Wright-Fisher simulations are performed using simulation.py. The output of these simulations is saved for processing. 

In this part, we use python code to run mpl.

Regularization value for individual part is 1 and for trait group part is 0.1.

In [2]:
importlib.reload(sim)

pdata = {
    'n_gen':    1000,                                   # number of generations
    'N':        1000,                                   # population size
    'mu':       2e-4,                                   # mutation rate
    'r':        2e-4,                                   # recombination rate
    't0':       0,                                      # start generation
    'T':        1000,                                   # final generation
    'ns':       [10, 20, 30, 40, 50,  80, 100, 1000],   # number of sequences to sample at each time point
    'dt':       [ 1,  5, 10, 20, 50],                   # spacing between generations
    'xpath':    'example',                              # input file path
    'xfile':    '0_ns1000_dt1',                         # input file name
    'n_ben':    10,                                     # number of beneficial mutations
    'n_neu':    30,                                     # number of neutral mutations
    'n_del':    10,                                     # number of deleterious mutations
    's_ben':    0.02,                                   # selection coefficient of beneficial mutations
    's_neu':    0,                                      # selection coefficient of neutral mutations
    's_del':    -0.02,                                  # selection coefficient of deleterious mutations
    's_tra':    0.1,                                    # trait coefficient
    'gamma':    1,                                      # regularization value
    'alphabet': ['A', 'T'],                             # all possible alleles                              
    'escape_group':  [[1,2,3],[11,12,13]],              # binary trait sites
    'save_cov': True,                                   # if true, save covariance matrix
    'example' : True,                                   # if true, this is an example, we will print the output
}

sim.simulate(**pdata)

Use 3 files to restore the information about trait groups. ("traitsites": trait sites , "traitseq": TF sequences for trait sites,"traitdis":distance between 2 neighboring trait sites)

In [3]:
'Create the necessary files'

escape_group = pdata['escape_group']

# trait distance 
f = open('%s/example/traitdis-0.dat'%(SIM_DIR), 'w')
for i in range(2):
    i_dis = []
    for j in range(2):
        i_dis.append(int(escape_group[i][j+1]-escape_group[i][j]))
    f.write('%s\n'%'\t'.join([str(ii) for ii in i_dis]))
f.close()

# trait sequence 
f = open('%s/example/traitseq-0.dat'%(SIM_DIR), 'w')
for i in range(2):
    f.write('%s\n'%'\t'.join([str(0) for j in range(3)]))
f.close()

# trait sites 
f = open('%s/example/traitsite-0.dat'%(SIM_DIR), 'w')
for i in range(2):
    f.write('%s\n'%'\t'.join([str(ii) for ii in escape_group[i]]))
f.close()

Run MPL with python code 

In [4]:
sim.run_mpl_binary(**pdata)

Execution time for simulation (binary case): 77.60822486877441 seconds
Calculation completes
inferred beneficial selection coefficients are:
[0.00922488 0.01779665 0.00704391 0.02181248 0.02952003 0.01376223
 0.01339774 0.01600949 0.01659001 0.01639045]
inferred neutral selection coefficients are:
[ 0.00148836 -0.01445969 -0.02071117 -0.04059499  0.00269984 -0.00856594
  0.00796704 -0.00208039  0.00028794 -0.00519714 -0.00063395 -0.00602022
  0.00929163 -0.00542618  0.00329977  0.00218841  0.0066797  -0.00267986
 -0.00518262  0.00279814  0.00216829 -0.02111263 -0.03461632  0.00192633
 -0.01030131  0.00231661 -0.00075496  0.00017662 -0.00798882 -0.01116452]
inferred deleterious selection coefficients are:
[-0.01220916 -0.00940609 -0.0244221  -0.04899712 -0.01576807 -0.01215628
 -0.0353718  -0.00930469 -0.02084199 -0.03503064]
inferred trait coefficients are:
[0.0907283  0.10474854]


Run MPL with C++ code. 
The result is similar to python code and much faster. The difference between 2 codes is partly because we do not use interpolation in python code.

In [5]:
'binary C code'
f = open("src/sim_example.sh",'w')
f.write('g++ binary/main.cpp binary/inf_binary.cpp binary/io_binary.cpp -march=native -lgsl -lgslcblas -o mpl\n')
f.write('./mpl -d ../data/simulation/example -i example-0_ns1000_dt1.dat ')
f.write('-o sc-0_ns1000_dt1-C.dat -g 1 -N 1e3 -mu 2e-4 -rr 2e-4 ')
f.write('-e traitsite-0.dat -es traitseq-0.dat -ed traitdis-0.dat -sc covariance-0_ns1000_dt1-C.dat\n')
f.close()


<a id='rep'></a>
### Repeating simulation
In this part, we run multiple simulations by Python. For all simulations, their initial conditions are the same, except for trait group, which is generated randomly.

In [6]:
import json
from importlib import reload
reload(sim)

SIM_DIR = 'data/simulation'
n_sim   = 100

pdata['xpath']    = 'jobs'
pdata['example']  = False
pdata['save_cov'] = False

# get random escape groups for 100 simulations
escape_groups = []
for n in range(n_sim):
    random_numbers   = random.sample(range(50), 6)
    escape_group_raw = [random_numbers[:3],random_numbers[3:]]
    escape_group     = [sorted(sublist) for sublist in escape_group_raw]
    escape_groups.append(escape_group)
    
    # trait sites 
    f = open('%s/jobs/traitsite/traitsite-%s.dat'%(SIM_DIR,n), 'w')
    for i in range(len(escape_group)):
        f.write('%s\n'%'\t'.join([str(ii) for ii in escape_group[i]]))
    f.close()
    
    # distance between 2 trait sites
    f = open('%s/jobs/traitdis/traitdis-%s.dat'%(SIM_DIR,n), 'w')
    for i in range(len(escape_group)):
        i_dis = []
        for j in range(len(escape_group[i])-1):
            i_dis.append(int(escape_group[i][j+1]-escape_group[i][j]))
        f.write('%s\n'%'\t'.join([str(ii) for ii in i_dis]))
    f.close()

# trait sequence 
f = open('%s/jobs/traitseq.dat'%(SIM_DIR), 'w')
for i in range(2):
    f.write('%s\n'%'\t'.join([str(0) for j in range(3)]))
f.close()
    
# save and load escape group information
with open("%s/jobs/escape_groups.dat"%SIM_DIR, 'w') as file:
    json.dump(escape_groups, file)

# with open("%s/jobs/escape_groups.dat"%SIM_DIR, 'r') as file:
#     escape_groups = json.load(file)

# inference
for n in range(n_sim):
    pdata['escape_group'] = escape_groups[n]
    pdata['xfile'] = 'example-'+str(n)+'_ns1000_dt1' 
    sim.simulate(**pdata)
#     sim.run_mpl_binary(**pdata)

print('we have done %d times simulations'%n_sim)

we have done 100 times simulations


Write down a shell code to run the results with C++

In [7]:
'binary C code'
f = open("src/sim.sh",'w')
f.write('g++ binary/main.cpp binary/inf_binary.cpp binary/io_binary.cpp -march=native -lgsl -lgslcblas -o mpl\n')
for n in range(n_sim):
    f.write('./mpl -d ../data/simulation/jobs -i sequences/example-%d_ns1000_dt1.dat '%n)
    f.write('-o output/sc-%d.dat -g 1 -N 1e3 -mu 2e-4 -rr 2e-4 '%n)
    f.write('-e traitsite/traitsite-%d.dat -es traitseq.dat -ed traitdis/traitdis-%d.dat '%(n,n))
    f.write('-sc covariance/covariance-%d.dat\n'%(n))
    
f.close()

print('we have produced the shell script run_sim.sh')

we have produced the shell script run_sim.sh


Create a csv file to store the results of all simulations.

In [8]:
nB = pdata['n_ben']
nD = pdata['n_del']
nN = pdata['n_neu']
nT = 2
seq_length = nB+nD+nN

f = open('%s/mpl_collected_C.csv'%SIM_DIR,'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
for i in range(nT):
    f.write(',tc_%d'%i)
f.write('\n')

for k in range(n_sim):
    sc = np.loadtxt('%s/jobs/output/sc-%d.dat'%(SIM_DIR,k))
    f.write('%d,1000,1'%(k))
    for ii in range(seq_length):
        f.write(',%f'%(sc[ii]))
    for ii in range(nT):
        f.write(',%f'%sc[-nT+ii])
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%n_sim)

collect all coefficients for 100 simulations


<a id='supp'></a>
### Supplement: no recombination term case
Run the results without recombination term by C++ 

In [9]:
'binary C code without recombination part'
f = open("src/sim_noR.sh",'w')
f.write('g++ binary/main.cpp binary/inf_binary.cpp binary/io_binary.cpp -march=native -lgsl -lgslcblas -o mpl\n')
for n in range(n_sim):
    f.write('./mpl -d ../data/simulation/jobs -i sequences/example-%d_ns1000_dt1.dat '%n)
    f.write('-o output_noR/sc-%d.dat -g 1 -N 1e3 -mu 2e-4 -rr 0 '%n)
    f.write('-e traitsite/traitsite-%d.dat -es traitseq.dat -ed traitdis/traitdis-%d.dat\n'%(n,n))
f.close()

print('we have produced the shell script sim_noR.sh')

we have produced the shell script sim_noR.sh


In [10]:
'binary C code without recombination part'

f = open('%s/mpl_collected_noR.csv'%SIM_DIR,'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
for i in range(nT):
    f.write(',tc_%d'%i)
f.write('\n')

for k in range(n_sim):
    sc = np.loadtxt('%s/jobs/output_noR/sc-%d.dat'%(SIM_DIR,k))
    f.write('%d,1000,1'%(k))
    for ii in range(seq_length):
        f.write(',%f'%(sc[ii]))
    for ii in range(nT):
        f.write(',%f'%sc[-nT+ii])
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations without recombination part'%n_sim)

collect all coefficients for 100 simulations without recombination part


<a id='nsdt'></a>
### Running the inference algorithms and compiling output
For one simulation, use different n_s and Δt to get the result

In [11]:
from importlib import reload
reload(sim)

ns_vals = pdata['ns']
dt_vals = pdata['dt']

g = open("src/sim_nsdt.sh",'w')
g.write('g++ binary/main.cpp binary/inf_binary.cpp binary/io_binary.cpp -march=native -lgsl -lgslcblas -o mpl\n')
for k in range(n_sim):
    pdata['xfile'] = 'example-'+str(k)
#     sim.py2c(**pdata)
    for i in range(len(ns_vals)):
        for j in range(len(dt_vals)):
            g.write('./mpl -d ../data/simulation/jobs ')
            g.write('-i sequences/nsdt/example-%d_ns%d_dt%d.dat '%(k,ns_vals[i],dt_vals[j]))
            g.write('-o output/nsdt/sc-%d_ns%d_dt%d.dat '%(k,ns_vals[i],dt_vals[j]))
            g.write('-g 1 -N 1e3 -mu 2e-4 -rr 2e-4 -e traitsite/traitsite-%d.dat '%(k))
            g.write('-es traitseq.dat -ed traitdis/traitdis-%d.dat\n'%(k))
g.close()

<a id='deal'></a>
### Dealing with the output
collect coefficients for all simulations and write the result into mpl_collected.csv.

In [12]:
nB = pdata['n_ben']
nD = pdata['n_del']
nN = pdata['n_neu']
nT = 2
seq_length = nB+nD+nN

n_sim = 100

f = open('%s/mpl_collected_nsdt.csv'%SIM_DIR,'w')
f.write('trajectory,t0,T,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
for i in range(nT):
    f.write(',tc_%d'%i)
f.write('\n')

for k in range(n_sim):
    for i in range(len(ns_vals)):
        for j in range(len(dt_vals)):
            sc = np.loadtxt('%s/jobs/output/nsdt/sc-%d_ns%d_dt%d.dat'%(SIM_DIR,k,ns_vals[i],dt_vals[j]))
            f.write('%d,0,1000,%d,%d'%(k,ns_vals[i],dt_vals[j]))
            for ii in range(seq_length):
                f.write(',%f'%sc[ii])
            for ii in range(nT):
                f.write(',%f'%sc[-nT+ii])
            f.write('\n')
f.close()

print('collect all coefficients for %d simulations'%(n_sim*len(ns_vals)*len(dt_vals)))

collect all coefficients for 4000 simulations


Calculate AUROC for beneficial and deleterious mutation and NRMSE for trait part\
Write these results into mpl_collected_extended.csv

In [13]:
nB = pdata['n_ben']
nD = pdata['n_del']
nN = pdata['n_neu']
nT = 2
fB = pdata['s_ben']
fD = pdata['s_del']
ft = pdata['s_tra']

true_ben = [1 if i in                        range(nB) else 0 for i in range(seq_length)];
true_del = [1 if i in range(seq_length-nD, seq_length) else 0 for i in range(seq_length)];
true_neu = [1 if i in range(        nB, seq_length-nD) else 0 for i in range(seq_length)];

coefs = ['sc_%d' % j for j in range(seq_length)]

df              = pd.read_csv('%s/mpl_collected_nsdt.csv'%SIM_DIR, memory_map=True);

# difference between inferred coefficients and true coefficients
for i in range(seq_length):
    if   true_ben[i]: df['d_sc%d' % i] = df['sc_%d' % i] - fB;
    elif true_del[i]: df['d_sc%d' % i] = df['sc_%d' % i] - fD;
    elif true_neu[i]: df['d_sc%d' % i] = df['sc_%d' % i];
for i in range(nT):
    df['d_tc%d' % i] = df['tc_%d' % i] - ft;

# AUROC for beneficial and deleterious mutation
df['AUROC_ben'] = pd.Series(data=[roc_auc_score(true_ben, np.array(df.iloc[i][coefs])) for i in range(len(df))]);
df['AUROC_del'] = pd.Series(data=[roc_auc_score(true_del,-np.array(df.iloc[i][coefs])) for i in range(len(df))]);

# error for trait part
norm  = nT*(ft**2)
error = 0
for i in range(nT):
    error += (df['tc_%d' % i] - ft) ** 2
df['error_tra'] = np.sqrt(error/norm)

df.to_csv('%s/mpl_collected_extended.csv'%SIM_DIR)

print('collect all AUROC and NRMSE for %d simulations'%(n_sim*len(ns_vals)*len(dt_vals)))

collect all AUROC and NRMSE for 4000 simulations


### Supplement: time-varying binary trait
Assume the selection coefficient for binary trait is a time-varying value instead of a constant value.

In [14]:
importlib.reload(sim)

generations = 1000
t1          = 200
t2          = generations - t1
fn          = np.zeros(generations+1)

for t in range(t1):
    fn[t] = 0.1/t1*t
for t in range(t1,generations+1):
    fn[t] = 0.1 - (0.1)/t2*(t-t1)
    
pdata = {
    'alphabet': ['A', 'T'],                             # all possible alleles                              
    'xpath':    'time-varying',                         # input file path
    'xfile':    '0_ns1000_dt1',                         # input file name
    'n_gen':    generations,                            # number of generations
    'N':        1000,                                   # population size
    'mu':       2e-4,                                   # mutation rate
    'r':        2e-4,                                   # recombination rate
    'n_ben':    10,                                     # number of beneficial mutations
    'n_neu':    30,                                     # number of neutral mutations
    'n_del':    10,                                     # number of deleterious mutations
    's_ben':    0.02,                                   # selection coefficient of beneficial mutations
    's_del':    -0.02,                                  # selection coefficient of deleterious mutations
    's_tra':    fn,                                     # trait coefficient
    'escape_group':  [[1,2,3],[11,12,13]],              # binary trait sites
    'inital_state': 4,                                  # initial states
}

In [15]:
n_sim   = 100

xpath = pdata['xpath']

# get random escape groups for 100 simulations
escape_groups = []
for n in range(n_sim):
    random_numbers   = random.sample(range(50), 6)
    escape_group_raw = [random_numbers[:3],random_numbers[3:]]
    escape_group     = [sorted(sublist) for sublist in escape_group_raw]
    escape_groups.append(escape_group)
    
    # trait sites 
    f = open('%s/%s/traitsite/traitsite-%s.dat'%(SIM_DIR,xpath,n), 'w')
    for i in range(len(escape_group)):
        f.write('%s\n'%'\t'.join([str(ii) for ii in escape_group[i]]))
    f.close()
    
    # distance between 2 trait sites
    f = open('%s/%s/traitdis/traitdis-%s.dat'%(SIM_DIR,xpath,n), 'w')
    for i in range(len(escape_group)):
        i_dis = []
        for j in range(len(escape_group[i])-1):
            i_dis.append(int(escape_group[i][j+1]-escape_group[i][j]))
        f.write('%s\n'%'\t'.join([str(ii) for ii in i_dis]))
    f.close()

# trait sequence 
f = open('%s/%s/traitseq.dat'%(SIM_DIR,xpath), 'w')
for i in range(2):
    f.write('%s\n'%'\t'.join([str(0) for j in range(3)]))
f.close()
    
# save and load escape group information
with open("%s/%s/escape_groups.dat"%(SIM_DIR,xpath), 'w') as file:
    json.dump(escape_groups, file)

# with open("%s/jobs/escape_groups.dat"%SIM_DIR, 'r') as file:
#     escape_groups = json.load(file)

# inference
for n in range(n_sim):
    pdata['escape_group'] = escape_groups[n]
    pdata['xfile'] = 'example-'+str(n)+'_ns1000_dt1' 
    sim.simulate_tv(**pdata)

print('we have done %d times simulations'%n_sim)

we have done 100 times simulations


In [16]:
'binary C code'
f = open("src/sim-tv.sh",'w')
f.write('g++ binary/main.cpp binary/inf_binary.cpp binary/io_binary.cpp -march=native -lgsl -lgslcblas -o mpl\n')
for n in range(n_sim):
    f.write('./mpl -d ../data/simulation/time-varying -i sequences/example-%d_ns1000_dt1.dat '%n)
    f.write('-o output/sc-%d.dat -g 1 -N 1e3 -mu 2e-4 -rr 2e-4 '%n)
    f.write('-e traitsite/traitsite-%d.dat -es traitseq.dat -ed traitdis/traitdis-%d.dat '%(n,n))
    f.write('-sc covariance/covariance-%d.dat\n'%(n))
    
f.close()

print('we have produced the shell script run_sim.sh')

we have produced the shell script run_sim.sh


In [17]:
'Create a csv file to store the results of all simulations.'

nB = pdata['n_ben']
nD = pdata['n_del']
nN = pdata['n_neu']
nT = len(pdata['escape_group'])
seq_length = nB+nD+nN

f = open('%s/mpl_collected_tv.csv'%SIM_DIR,'w')
f.write('trajectory,ns,delta_t')
for i in range(seq_length):
    f.write(',sc_%d'%i)
for i in range(nT):
    f.write(',tc_%d'%i)
f.write('\n')

for k in range(n_sim):
    sc = np.loadtxt('%s/time-varying/output/sc-%d.dat'%(SIM_DIR,k))
    f.write('%d,1000,1'%(k))
    for ii in range(seq_length):
        f.write(',%f'%(sc[ii]))
    for ii in range(nT):
        f.write(',%f'%sc[-nT+ii])
    f.write('\n')
f.close()

print('collect all coefficients for %d simulations with time-varying trait coefficients'%n_sim)

collect all coefficients for 100 simulations with time-varying trait coefficients
